In [40]:
#!/usr/bin/env python3

import pandas as pd
import errno
import os, sys
import numpy as np

'''
Author: Enrico Ceccolini
    Given the raw data of Eurora (CORE_MEASUREMENT_FULL)
    and a time interval, it returns only the measurements
    of that period, sampled at 1 minute intervals
'''



datadir = "/datasets/eurora_data/db/"

# settings
global_start_time = pd.to_datetime('2014-03-24')
global_end_time = pd.to_datetime('2015-08-12')

#interval_comment = "Andrea"
#start_time = pd.to_datetime('2014-03-31')
#end_time = pd.to_datetime('2014-05-01')

cpu = '07'
infile = datadir + "CPUs/cpu" + cpu + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("there are {} jobs in the whole dataset".format(whole_node_power_data.shape[0]))

there are 6871488 jobs in the whole dataset


In [41]:
whole_node_power_data.head(1)

,07,cpu_id,timestamp,pow_cpu,pow_dram,pow_pkg
0,7,0,2014-03-24 13:56:27,2.64898,2.805525,18.56028


In [42]:
whole_node_power_data = whole_node_power_data.drop([str(cpu)], axis=1)

In [43]:
# drop useless columns
# whole_node_power_data = whole_node_power_data.drop(['01'], axis=1)

I notice that there are duplicated rows (same timestamp and cpu_id, quite strange).

In [44]:
# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates()
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))

data rows after drop_duplicates 6797660


drop the rows before the selected started time

In [45]:
# select data in the specified interval
#interval_data = whole_node_power_data[pd.to_datetime(whole_node_power_data['timestamp']) >= start_time]
#interval_data = interval_data[pd.to_datetime(interval_data['timestamp']) <= end_time]
#print("data row from {} to {}: {}".format(start_time, end_time, interval_data.shape[0]))

In [46]:
print(cpu)
infile = datadir + "CPUs/cpu" + cpu + ".csv"

# read dataset
whole_node_power_data = pd.read_csv(infile) # Read a comma-separated values (csv) file into DataFrame
print("data rows {}".format(whole_node_power_data.shape))

# drop useless columns
whole_node_power_data = whole_node_power_data.drop([str(cpu)], axis=1)

# drop the duplicates rows (same timestamp, cpu_id, ...)
whole_node_power_data = whole_node_power_data.drop_duplicates() 
print("data rows after drop_duplicates {}".format(whole_node_power_data.shape[0]))

07
data rows (6871488, 6)
data rows after drop_duplicates 6797660


In [47]:
# select data in the specified interval
#interval_data = whole_node_power_data.loc[(pd.to_datetime(whole_node_power_data['timestamp']) >= start_time) & (pd.to_datetime(whole_node_power_data['timestamp']) <= end_time)]
#print("data row from {} to {}: {}".format(start_time, end_time, interval_data.shape[0]))
# no more interval
interval_data = whole_node_power_data

create two different dataFrame. One for cpu0 and one for cpu1

In [48]:
# split the data referring to the cpu0 and cpu0
cpu0 = interval_data[interval_data['cpu_id'] == 0]
cpu1 = interval_data[interval_data['cpu_id'] == 1]
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3398798
data row after split: 3398862


In [49]:
# drop the duplicates rows 
cpu0 = cpu0.drop_duplicates(subset=['timestamp'])
cpu1 = cpu1.drop_duplicates(subset=['timestamp'])
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3314171
data row after split: 3314163


In [50]:
if cpu0.shape[0] < cpu1.shape[0]:
    cpu1 = cpu1[cpu1.timestamp >= cpu0['timestamp'].iloc[0]]
if cpu1.shape[0] < cpu0.shape[0]:
    cpu0 = cpu0[cpu0.timestamp >= cpu1['timestamp'].iloc[0]]
print("data row after split: {}".format(cpu0.shape[0]))
print("data row after split: {}".format(cpu1.shape[0]))

data row after split: 3314171
data row after split: 3314163


In [51]:
cpu1.head(1)

,cpu_id,timestamp,pow_cpu,pow_dram,pow_pkg
12684,1,2014-03-24 13:56:27,2.725998,2.47114,18.481787


convert the timestamp column to the pd datetime format

In [52]:
# convert the timestamp column to the Pandas timestamp format
cpu0.timestamp = pd.to_datetime(cpu0.timestamp)
cpu1.timestamp = pd.to_datetime(cpu1.timestamp)
cpu0.head(1)

,cpu_id,timestamp,pow_cpu,pow_dram,pow_pkg
0,0,2014-03-24 13:56:27,2.64898,2.805525,18.56028


In [53]:
cpu1.head(1)

,cpu_id,timestamp,pow_cpu,pow_dram,pow_pkg
12684,1,2014-03-24 13:56:27,2.725998,2.47114,18.481787


In [54]:
#cpu0 = cpu0.append({'timestamp' : global_start_time} , ignore_index=True)
#cpu0.tail(1)

In [55]:
cpu0 = cpu0.set_index(["timestamp"])
cpu1 = cpu1.set_index(["timestamp"])
cpu0.head(5)

,cpu_id,pow_cpu,pow_dram,pow_pkg
timestamp,,,,
2014-03-24 13:56:27,0,2.648980,2.805525,18.560280
2014-03-24 13:56:32,0,2.759270,2.907987,18.762611
2014-03-24 13:56:37,0,2.741546,2.852764,18.723516
2014-03-24 13:56:42,0,2.621199,2.770980,18.507101
2014-03-24 13:56:47,0,2.645240,2.721337,18.551910


In [56]:
# resample data from 5 sec to 1 min, using the mean for the new values 
cpu0_1min = cpu0.resample('1Min').mean()
print("data rows {}".format(cpu0_1min.shape[0]))

data rows 561887


In [57]:
cpu1_1min = cpu1.resample('1Min').mean()
print("data rows {}".format(cpu1_1min.shape[0]))

data rows 561887


In [58]:
cpu0_1min.head(1)

,cpu_id,pow_cpu,pow_dram,pow_pkg
timestamp,,,,
2014-03-24 13:56:00,0.0,2.69024,2.798769,18.631529


In [59]:
cpu1_1min.head(1)

,cpu_id,pow_cpu,pow_dram,pow_pkg
timestamp,,,,
2014-03-24 13:56:00,1.0,2.761196,2.439045,18.549075


In [60]:
cpu0_1min = cpu0_1min.drop(['cpu_id'], axis=1)
cpu1_1min = cpu1_1min.drop(['cpu_id'], axis=1)

In [61]:
cpu0_1min.head(1)

,pow_cpu,pow_dram,pow_pkg
timestamp,,,
2014-03-24 13:56:00,2.69024,2.798769,18.631529


In [62]:
first_date = cpu0_1min.index[0]
last_date = cpu0_1min.index[cpu0_1min.shape[0]-1]

In [63]:
index_before = pd.date_range(start=global_start_time, end=first_date - np.timedelta64(1, 'm'), freq='1Min')
index_before

DatetimeIndex(['2014-03-24 00:00:00', '2014-03-24 00:01:00',
               '2014-03-24 00:02:00', '2014-03-24 00:03:00',
               '2014-03-24 00:04:00', '2014-03-24 00:05:00',
               '2014-03-24 00:06:00', '2014-03-24 00:07:00',
               '2014-03-24 00:08:00', '2014-03-24 00:09:00',
               ...
               '2014-03-24 13:46:00', '2014-03-24 13:47:00',
               '2014-03-24 13:48:00', '2014-03-24 13:49:00',
               '2014-03-24 13:50:00', '2014-03-24 13:51:00',
               '2014-03-24 13:52:00', '2014-03-24 13:53:00',
               '2014-03-24 13:54:00', '2014-03-24 13:55:00'],
              dtype='datetime64[ns]', length=836, freq='T')

In [64]:
index_after = pd.date_range(start=last_date + np.timedelta64(1, 'm'), end=global_end_time, freq='1Min')
index_after

DatetimeIndex(['2015-04-18 18:43:00', '2015-04-18 18:44:00',
               '2015-04-18 18:45:00', '2015-04-18 18:46:00',
               '2015-04-18 18:47:00', '2015-04-18 18:48:00',
               '2015-04-18 18:49:00', '2015-04-18 18:50:00',
               '2015-04-18 18:51:00', '2015-04-18 18:52:00',
               ...
               '2015-08-11 23:51:00', '2015-08-11 23:52:00',
               '2015-08-11 23:53:00', '2015-08-11 23:54:00',
               '2015-08-11 23:55:00', '2015-08-11 23:56:00',
               '2015-08-11 23:57:00', '2015-08-11 23:58:00',
               '2015-08-11 23:59:00', '2015-08-12 00:00:00'],
              dtype='datetime64[ns]', length=165918, freq='T')

In [65]:
padding_before = pd.DataFrame(index=index_before, columns=['pow_cpu', 'pow_dram', 'pow_pkg'])
padding_before.index.name = 'timestamp'
#padding_before

In [66]:
padding_after = pd.DataFrame(index=index_after, columns=['pow_cpu', 'pow_dram', 'pow_pkg'])
padding_after.index.name = 'timestamp'
# padding_after

In [67]:
# convert the timestamp column to the Pandas timestamp format
#cpu0.timestamp = pd.to_datetime(cpu0.timestamp)
#cpu1.timestamp = pd.to_datetime(cpu1.timestamp)

In [68]:
cpu0_1min = pd.concat([padding_before, cpu0_1min])
cpu1_1min = pd.concat([padding_before, cpu1_1min])
cpu0_1min = pd.concat([cpu0_1min, padding_after])
cpu1_1min = pd.concat([cpu1_1min, padding_after])

In [69]:
cpu0_1min

,pow_cpu,pow_dram,pow_pkg
timestamp,,,
2014-03-24 00:00:00,NaN,NaN,NaN
2014-03-24 00:01:00,NaN,NaN,NaN
2014-03-24 00:02:00,NaN,NaN,NaN
2014-03-24 00:03:00,NaN,NaN,NaN
2014-03-24 00:04:00,NaN,NaN,NaN
2014-03-24 00:05:00,NaN,NaN,NaN
2014-03-24 00:06:00,NaN,NaN,NaN
2014-03-24 00:07:00,NaN,NaN,NaN
2014-03-24 00:08:00,NaN,NaN,NaN


In [70]:
cpu0_1min = cpu0_1min.rename(index=str, columns={"pow_cpu": "pow_cpu_0", "pow_dram": "pow_dram_0", "pow_pkg": "pow_pkg_0"})
cpu1_1min = cpu1_1min.rename(index=str, columns={"pow_cpu": "pow_cpu_1", "pow_dram": "pow_dram_1", "pow_pkg": "pow_pkg_1"})
# cpu0_1min.head(1)

In [71]:
# concat
node_1min = pd.concat([cpu0_1min, cpu1_1min], axis=1)

In [72]:
node_1min.head(1)

,pow_cpu_0,pow_dram_0,pow_pkg_0,pow_cpu_1,pow_dram_1,pow_pkg_1
timestamp,,,,,,
2014-03-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
node_1min['pow_tot_0'] = cpu0_1min['pow_cpu_0'] + cpu0_1min['pow_dram_0'] + cpu0_1min['pow_pkg_0'] 
node_1min['pow_tot_1'] = cpu1_1min['pow_cpu_1'] + cpu1_1min['pow_dram_1'] + cpu1_1min['pow_pkg_1']

In [74]:
node_1min.head(1)

,pow_cpu_0,pow_dram_0,pow_pkg_0,pow_cpu_1,pow_dram_1,pow_pkg_1,pow_tot_0,pow_tot_1
timestamp,,,,,,,,
2014-03-24 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
node_1min = node_1min.drop(['pow_cpu_0', 'pow_dram_0', 'pow_pkg_0', 'pow_cpu_1', 'pow_dram_1', 'pow_pkg_1'], axis=1)

In [76]:
node_1min.head(1)

,pow_tot_0,pow_tot_1
timestamp,,
2014-03-24 00:00:00,NaN,NaN


In [77]:
pow_tot_1 = cpu0_1min['pow_cpu_0'] + cpu0_1min['pow_dram_0'] + cpu0_1min['pow_pkg_0']
pow_tot_2 = cpu1_1min['pow_cpu_1'] + cpu1_1min['pow_dram_1'] + cpu1_1min['pow_pkg_1']
print("first: {}, second: {}".format(pow_tot_1.shape, pow_tot_2.shape))

first: (728641,), second: (728641,)


In [41]:
# write
#outfile = datadir + 'CPUs/' + interval_comment + "/node" + cpu + "_1min_" + interval_comment + ".csv"
#node_1min.to_csv(outfile)